In [1]:
import re
import requests
import pandas as pd

In [2]:
page_all_results = requests.get('https://www.parkrun.ru/kuzminki/results/eventhistory/', 
                                headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86.0) Gecko/20100101 Firefox/86.0'})

In [93]:
data = pd.read_html(page_all_results.text)[0]
data = data.drop(columns=data.columns[8:])

In [94]:
data = data.rename(columns={data.columns[0]: 'n', data.columns[1]: 'date',
                           data.columns[2]: 'finishers', data.columns[3]:'volunteers',
                           data.columns[4]: 'men_name', data.columns[5]: 'men_time',
                           data.columns[6]: 'women_name', data.columns[7]: 'women_time'})

In [95]:
pd.set_option('display.max_colwidth', None)

In [96]:
data[data.columns[1]] = data[data.columns[1]].transform(lambda s: re.search(r'\d\d/\d\d/\d\d', s)[0])
data[data.columns[2]] = data[data.columns[2]].transform(lambda s: s.replace('finishers', ''))
data[data.columns[3]] = data[data.columns[3]].transform(lambda s: s.replace('volunteers', ''))
data[data.columns[4]] = data[data.columns[4]].transform(lambda s: re.sub(r'\d\d:\d\d:\d\d', '', s))
data[data.columns[6]] = data[data.columns[6]].transform(lambda s: re.sub(r'\d\d:\d\d:\d\d', '', s))

In [97]:
data

,n,date,finishers,volunteers,men_name,men_time,women_name,women_time
0,203,14/03/20,154,27,Виктор НИКОЛАЕВ,00:15:46,Наталья СЕЛЕЗНЕВА,00:20:13
1,202,07/03/20,176,37,Дмитрий ТИХОНОВ,00:17:53,Наталья СЕЛЕЗНЕВА,00:20:15
2,201,29/02/20,185,35,Герман ЖАРОВ,00:18:22,Наталья СЕЛЕЗНЕВА,00:20:35
3,200,22/02/20,246,32,Дмитрий ТИХОНОВ,00:17:37,Екатерина Лицоева,00:22:27
4,199,15/02/20,128,24,Юрий ВОЛОДИН,00:17:38,Светлана БЕЛАВИНА,00:22:42
...,...,...,...,...,...,...,...,...
198,5,16/07/20,42,12,Дмитрий БОНДАРЬ,00:18:36,Светлана КАНТОНИСТОВА,00:22:32
199,4,09/07/20,46,13,Ринат АВЗЕЛИТДИНОВ,00:18:05,Ольга АКИМОВА,00:23:14
200,3,02/07/20,27,10,Дмитрий БОНДАРЬ,00:18:52,Янчик КРЖИЖАНОВСКАЯ,00:23:24
201,2,25/06/20,40,15,Алексей КУХАРЕНКО,00:18:28,Olga LAZAREVA,00:22:15


In [98]:
data[data.columns[5]] = data[data.columns[5]].apply(pd.to_timedelta)
data[data.columns[7]] = data[data.columns[7]].apply(pd.to_timedelta)

In [99]:
mens_res = data.drop(data.iloc[:,2:4], axis=1).drop(data.iloc[:,6:], axis=1)
woman_res = data.drop(data.iloc[:,2:6], axis=1)

## Среднее время победителя (мужчины)

In [100]:
mean_mens_time = mens_res[mens_res.columns[3]].mean().components
print(f'{mean_mens_time.minutes}:{mean_mens_time.seconds:02d}')

17:48


## Топ10 мужских результатов

In [101]:
data.sort_values(by=[data.columns[5]]).drop(data.iloc[:,2:4], axis=1).drop(data.iloc[:,6:], axis=1).reset_index(drop=True).head(10)

,n,date,men_name,men_time
0,107,09/06/20,Иван БУБНОВ,0 days 00:15:43
1,203,14/03/20,Виктор НИКОЛАЕВ,0 days 00:15:46
2,54,17/06/20,Виктор НИКОЛАЕВ,0 days 00:15:51
3,100,21/04/20,Виктор НИКОЛАЕВ,0 days 00:15:59
4,162,15/06/20,Герман ЖАРОВ,0 days 00:16:07
5,179,12/10/20,Герман ЖАРОВ,0 days 00:16:09
6,67,16/09/20,Дима СИЗОВ,0 days 00:16:10
7,19,22/10/20,Герман ЖАРОВ,0 days 00:16:24
8,172,24/08/20,Герман ЖАРОВ,0 days 00:16:31
9,130,17/11/20,Александр БОСКИН,0 days 00:16:33


## Среднее время победителя (женщины)

In [102]:
mean_woman_time = woman_res[woman_res.columns[3]].mean().components
print(f'{mean_woman_time.minutes}:{mean_woman_time.seconds:02d}')

23:00


## Топ10 женских результатов

In [103]:
data.sort_values(by=[data.columns[7]]).drop(data.iloc[:,2:6], axis=1).reset_index(drop=True).head(10)

,n,date,women_name,women_time
0,68,23/09/20,Тома СИЗОВА,0 days 00:18:16
1,67,16/09/20,Тома СИЗОВА,0 days 00:18:44
2,50,20/05/20,Алсу ШАГИЕВА,0 days 00:18:47
3,14,17/09/20,Алсу ШАГИЕВА,0 days 00:18:53
4,1,18/06/20,Алсу ШАГИЕВА,0 days 00:18:57
5,48,06/05/20,Алсу ШАГИЕВА,0 days 00:19:02
6,72,21/10/20,Светлана КИРЕЕВА,0 days 00:19:08
7,20,29/10/20,Nina ZARINA,0 days 00:19:08
8,175,14/09/20,Екатерина БАШМАКОВА,0 days 00:19:13
9,100,21/04/20,Алсу ШАГИЕВА,0 days 00:19:13
